In [67]:
!pip install -qU pypdf
!pip install -qU langchain-openai
!pip install langchain_community
!pip install faiss-cpu
!pip install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [68]:
#경로 설정
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

#모델 설정
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9)

In [70]:
#pdf 읽어오기
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/dajung2140/work/kosa-chatgpt-2025-1st/src/exercise/dawoo/day01/doc/소나기.pdf")
documents = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

7

In [71]:
docs[:3]

[Document(metadata={'source': '/home/dajung2140/work/kosa-chatgpt-2025-1st/src/exercise/dawoo/day01/doc/소나기.pdf', 'page': 0, 'page_label': '1'}, page_content='- 1 -\n소나기황순원소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯이.벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다.요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.다음 날은 좀 늦게 개울가로 나왔다.이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 것이리라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사람이 있어야 길을 비킬 모양이다.그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다. 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.다 건너가더니만 홱 이리로 돌아서며,“이 바보.”조약돌이 날아왔다.소년은 저도 모르게 벌떡 일어섰다.단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐.이제 저쯤 갈밭머리로 소녀가 나타나리라. 꽤 오랜 시간이 지났다고 생각됐다. 그런데도 소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.저 쪽 갈밭머리

In [72]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    ).from_loaders([loader])

index.vectorstore.save_local("faiss-vectorstore")

In [73]:
index

vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f3f73b08400>


In [74]:
# from langchain_openai import OpenAIEmbeddings
# from langchain_core.vectorstores import InMemoryVectorStore

# embeddings = OpenAIEmbeddings()

# vector_store = InMemoryVectorStore.from_documents(pages, embeddings)
# docs = vector_store.similarity_search("징검다리", k=3)

# for doc in docs:
#     print(f'Page : {doc.page_content[:300]}\n')

In [75]:
chat1 = index.query("소년의 심정 변화는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.
Traceback (most recent call last):
  File "/home/dajung2140/.vscode-server/extensions/ms-python.python-2024.22.2-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
  File "/home/dajung2140/.local/lib/python3.10/site-packages/langchain/indexes/vectorstore.py", line 38, in query
    raise NotImplementedError(
NotImplementedError: This API has been changed to require an LLM. Please provide an llm to use for querying the vectorstore.
For example,
from langchain_openai import OpenAI
llm = OpenAI(temperature=0)



In [ ]:
chat2 = index.query("소년의 심정 변화는?", llm=chat)

In [ ]:
print(chat2)
print(type(chat2))

소년의 심정 변화는 소녀가 이사 간다는 사실을 알게 된 후부터 시작된다. 처음에는 소녀의 이사 소식이 그다지 슬프지 않다고 느끼지만, 시간이 지나면서 소녀와 함께했던 순간들을 회상하고 그녀와의 이별에 대한 아쉬움을 느끼게 된다. 소녀를 생각하며 혼잣말을 하면서 점점 더 그리움과 안타까움을 느끼고, 소녀에게 맛있는 호두를 먹여주고 싶다는 마음도 생긴다. 결국 소년은 소녀에게 전하고 싶었던 말을 하지 못한 것을 후회하며, 그리움과 따뜻한 감정으로 가득 차게 된다. 소녀와의 소중한 순간들과 이별의 아쉬움이 얽히면서 소년의 감정은 복잡하게 변화하게 된다.
<class 'str'>


In [ ]:
os.getcwd()